In [1]:
# load the packages
import os
import requests
import warnings
import datetime as dt
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [2]:
# helper functions
def get_key():
    "Get the API key from an .env file"
    if ".env" not in os.listdir("../"):
        print("Configuring API Key...")
        key = input("Amberdata API Key: ")
        with open("../.env", "w") as f:
            f.write(f"AMBERDATA_API_KEY={key}\n")
    load_dotenv(verbose=True)
    return {
        "AMBERDATA_API_KEY": os.getenv("AMBERDATA_API_KEY")
    }

def get_response(url, headers=None, queryString=None):
    "Get the REST response from the specified URL"
    if not headers:
        headers = {'x-api-key': api_key["AMBERDATA_API_KEY"]}
    if queryString:
        response = requests.request("GET", url, headers=headers, params=queryString)
    else:
        response = requests.request("GET", url, headers=headers)
    response = json.loads(response.text)
    try:
        if response["title"] == "OK":
            return response["payload"]
    except Exception:
        return None
    
api_key = get_key()

In [6]:
url = "https://web3api.io/api/v2/transactions/"

querystring = {"status":"pending","size":"100"}

headers = {
    'x-amberdata-blockchain-id': "bitcoin-mainnet",
    'x-api-key': api_key["AMBERDATA_API_KEY"]
    }

response = get_response(url, headers, querystring)

In [7]:
response

{'records': [{'blockNumber': None,
   'confirmedAt': None,
   'confirmedHash': None,
   'createdAt': '2020-07-29T00:49:12.372Z',
   'from': [{'address': '181FkF8XG67Ra3fxQg2CjhVNZBqVq4wAWS'}],
   'hash': '2b61206e83faf579340f0bc8be08c131e5b9001a082d1b25f37d21e1ea763c5c',
   'index': 0,
   'occurrences': 20,
   'state': 1,
   'to': [{'address': '371soNkLGJzGAJAt2VYRv6jjwCneFFpETW'}],
   'updatedAt': '2020-07-29T00:49:12.371Z',
   'value': '1163252',
   'inputs': [{'addresses': ['181FkF8XG67Ra3fxQg2CjhVNZBqVq4wAWS'],
     'index': 0,
     'meta': '{}',
     'requiredSignatures': 1,
     'scriptAsm': '30450221009ec824e60cd677ece80145cd9da2a7f13d69793294701b7999ead34d2626f22e02203558de230b07a36118bb6ef26cd68b2079f2a301e2d910432bd26274a82a6630[ALL] 0214c083dd48fe03cbc17ccbce58d9d2160d7dd8efe9548bf28eb3cf8c6dcdae08',
     'scriptHex': '4830450221009ec824e60cd677ece80145cd9da2a7f13d69793294701b7999ead34d2626f22e02203558de230b07a36118bb6ef26cd68b2079f2a301e2d910432bd26274a82a663001210214c083dd

In [11]:
response["records"][0].keys()

dict_keys(['blockNumber', 'confirmedAt', 'confirmedHash', 'createdAt', 'from', 'hash', 'index', 'occurrences', 'state', 'to', 'updatedAt', 'value', 'inputs', 'outputs', 'statusResult', 'timestamp'])